In [2]:
import gym
import copy
import random
import torch
import torchvision.transforms as T
from torch.autograd import Variable
import numpy as np
import cv2

In [2]:
class DQN():
    ''' Deep Q Neural Network class. '''
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=0.05):
        self.env_name = env_name       
        self.env = gym.make(env_name)
        
        # State variables
        self.ROWS = 160
        self.COLS = 240
        self.REM_STEP = 4
        self.EPISODES = 10
            
        # Model variables    
        self.criterion = torch.nn.MSELoss()
        self.model = torch.nn.Sequential(
                        torch.nn.Linear(state_dim, hidden_dim),
                        torch.nn.LeakyReLU(),
                        torch.nn.Linear(hidden_dim, hidden_dim*2),
                        torch.nn.LeakyReLU(),
                        torch.nn.Linear(hidden_dim*2, action_dim)
                    )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

        # Greedy variables
        self.epsilon = 0.05
        self.decay_rate = 0.99
        
        
# Model functions
    def update(self, state, y):
        """Update the weights of the network given a training sample. """
        y_pred = self.model(torch.Tensor(state))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def predict(self, state):
        """ Compute Q values for all actions using the DQL. """
        with torch.no_grad():
            return self.model(torch.Tensor(state))

        

        
# State and Image functions
    def imshow(self, image, rem_step=0):
        cv2.imshow(env_name+str(rem_step), image[rem_step,...])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            return
       
    def GetImage(self):
        img = self.env.render(mode='rgb_array')
  
        img_rgb = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_rgb_resized = cv2.resize(img_rgb, (self.COLS, self.ROWS), interpolation=cv2.INTER_CUBIC)
        img_rgb_resized[img_rgb_resized < 255] = 0
        img_rgb_resized = img_rgb_resized / 255

        self.image_memory = np.roll(self.image_memory, 1, axis = 0)
        self.image_memory[0,:,:] = img_rgb_resized
        
        return np.expand_dims(self.image_memory, axis=0)
    
    def reset(self):
        self.env.reset()
        for i in range(self.REM_STEP):
            state = self.GetImage()
        return state

    def step(self,action):
        next_state, reward, done, info = self.env.step(action)
        next_state = self.GetImage()
        return next_state, reward, done, info

# Action function
    def act(self,state):
        if random.random() < self.epsilon:
            action = env.action_space.sample()
        else:
            q_values = model.predict(state)
            action = torch.argmax(q_values).item()
        return action
    
# Main loop
    def run(self):
        for episode in range(self.EPISODES):
            state = self.reset()
            # Get_state
            for t in range(500):               
                # Select action
                action = self.act(state)
                
                
        

                next_state, reward, done, info = self.step(action)
                

                if done:
                    break
            self.epsilon = max(self.epsilon * self.eps_decay, 0.05)

if __name__ == "__main__":
    env_name = 'CartPole-v1'
    agent = DQN_CNN_Agent(env_name)
    agent.run()